	1.	EmbeddingDataset 클래스: .npy 파일을 불러와 임베딩과 레이블을 분리하고, 임베딩 벡터의 평균을 사용해 하나의 벡터로 변환했습니다.
	2.	EfficientNet 모델: EfficientNet을 기반으로 하여 마지막 레이어를 이진 분류용으로 수정했습니다.
	3.	학습 및 검증 루프: 각 에포크마다 학습 손실(train_loss), 검증 손실(val_loss), 정확도(accuracy)를 계산합니다.
    4.	임베딩을 4D로 변환: X.view(1, 16, 32)를 통해 임베딩을 (1, 16, 32) 형식으로 변경했습니다.
	5.	EfficientNet 첫 Conv 레이어 수정: 첫 번째 Conv 레이어의 입력 채널을 1로 설정하여, 단일 채널 2D 입력을 처리할 수 있도록 수정했습니다.

In [4]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision.models import efficientnet_b0
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# MPS 장치 확인
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device for training: {device}")

# 경로 설정
embedding_folder = '/Users/gible/오픈소스sw/Train_Data/train_data/Embedding_data'

class EmbeddingDataset(Dataset):
    def __init__(self, embedding_folder):
        self.files = [os.path.join(embedding_folder, file) for file in os.listdir(embedding_folder) if file.endswith('.npy')]

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        data = np.load(self.files[idx])
        X = torch.tensor(data, dtype=torch.float32).mean(dim=0)  # 512차원 평균 벡터 생성
        X = X.view(1, 16, 32)  # (512,) -> (1, 16, 32) 형태로 변환하여 4D 텐서로 만듦
        y = torch.tensor(1 if "REAL" in self.files[idx] else 0, dtype=torch.long)  # 예시 라벨 처리
        return X, y


# 데이터셋 로드 및 분할
dataset = EmbeddingDataset(embedding_folder)
train_indices, val_indices = train_test_split(range(len(dataset)), test_size=0.2, random_state=42)
train_subset = torch.utils.data.Subset(dataset, train_indices)
val_subset = torch.utils.data.Subset(dataset, val_indices)

train_loader = DataLoader(train_subset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_subset, batch_size=16, shuffle=False)

# 2. 모델 정의 (EfficientNet 사용)

# 모델 정의 (EfficientNet 사용)
model = efficientnet_b0(pretrained=True)
model.features[0][0] = nn.Conv2d(1, model.features[0][0].out_channels, kernel_size=3, stride=2, padding=1, bias=False)  # 첫 Conv 레이어 수정
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 2)  # 이진 분류
model = model.to(device)

# 손실 함수 및 옵티마이저 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# 3. 학습 및 검증 루프
num_epochs = 5
for epoch in range(num_epochs):
    # 학습 모드
    model.train()
    train_loss = 0.0
    train_preds, train_labels = [], []

    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        train_preds.extend(preds.cpu().numpy())
        train_labels.extend(y_batch.cpu().numpy())

    train_accuracy = accuracy_score(train_labels, train_preds)
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss/len(train_loader):.4f}, Train Accuracy: {train_accuracy:.4f}")

    # 검증 모드
    model.eval()
    val_loss = 0.0
    val_preds, val_labels = [], []

    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            val_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            val_preds.extend(preds.cpu().numpy())
            val_labels.extend(y_batch.cpu().numpy())

    val_accuracy = accuracy_score(val_labels, val_preds)
    print(f"Validation Loss: {val_loss/len(val_loader):.4f}, Validation Accuracy: {val_accuracy:.4f}")

Using device for training: mps
Epoch [1/5], Train Loss: 0.5733, Train Accuracy: 0.7111
Validation Loss: 0.0000, Validation Accuracy: 1.0000
Epoch [2/5], Train Loss: 0.3945, Train Accuracy: 0.8762
Validation Loss: 0.0016, Validation Accuracy: 1.0000
Epoch [3/5], Train Loss: 0.2760, Train Accuracy: 0.9651
Validation Loss: 0.0000, Validation Accuracy: 1.0000
Epoch [4/5], Train Loss: 0.2216, Train Accuracy: 0.9873
Validation Loss: 0.3785, Validation Accuracy: 0.7342
Epoch [5/5], Train Loss: 0.1692, Train Accuracy: 0.9968
Validation Loss: 0.2897, Validation Accuracy: 0.9367


	•	과적합 가능성: 학습 데이터에 대한 성능은 거의 완벽하지만, 검증 데이터에서 성능이 약간 떨어지는 것으로 보아 과적합이 발생했을 가능성이 있습니다.
	•	모델 개선 방향:
	•		과적합을 줄이기 위해 드롭아웃(dropout) 레이어 추가나 L2 정규화(weight decay) 등의 기법을 시도해 볼 수 있습니다.
	•		학습 에포크 수를 줄이거나, 학습률(learning rate)을 조정해 보는 것도 도움이 될 수 있습니다.
